# Loading Set

In [ ]:
from keras import losses, metrics
from keras.models import Sequential
from keras.models import Model, load_model
from keras.optimizers import Adam as Adam
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D, BatchNormalization, Dropout, Activation
from keras.layers import GlobalAveragePooling1D, Flatten, SeparableConv1D, Input, LSTM, GRU, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve
import tensorflow as tf
import os, sys, pickle
import pandas as pd
import random

In [ ]:
print('loading train...', flush=True, end='')

# x를 loading해서 (batch_size, step, channel)
input_path = '../dataset/preprocess4/welch/'
x_train = np.load(input_path+'x_train.npz', allow_pickle=True)['arr_0']
x_test = np.load(input_path+'x_test.npz', allow_pickle=True)['arr_0']
x_val = np.load(input_path+'x_val.npz', allow_pickle=True)['arr_0']
y_train = np.load(input_path+'y_train.npz')['arr_0']
y_test = np.load(input_path+'y_test.npz')['arr_0']
y_val = np.load(input_path+'y_val.npz')['arr_0']
print('done', flush=True)


x_train_ecg = x_train[:,:,:,1]
x_val_ecg = x_val[:,:,:,1]
x_test_ecg = x_test[:,:,:,1]


print('x_train shape:', x_train_ecg.shape)
print('x_test.shape:', x_test_ecg.shape)
print('x_val.shape:', x_val_ecg.shape)


In [ ]:
print('loading train...', flush=True, end='')

# x를 loading해서 (batch_size, step, channel)
input_path = '../dataset/preprocess6/welch1/'
x_train = np.load(input_path+'x_train_pacu.npz', allow_pickle=True)['arr_0']
x_test = np.load(input_path+'x_test_pacu.npz', allow_pickle=True)['arr_0']
x_val = np.load(input_path+'x_val_pacu.npz', allow_pickle=True)['arr_0']
y_train = np.load(input_path+'y_train_pacu.npz')['arr_0']
y_test = np.load(input_path+'y_test_pacu.npz')['arr_0']
y_val = np.load(input_path+'y_val_pacu.npz')['arr_0']
print('done', flush=True)


x_train2 = np.ones(shape=x_train.shape)
for idx, x in enumerate(x_train):
    x_train2[idx,:,:,0] = pd.DataFrame(x[:,:,0]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
    x_train2[idx,:,:,1] = pd.DataFrame(x[:,:,1]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values


x_val2 = np.ones(shape=x_val.shape)
for idx, x in enumerate(x_val):
    x_val2[idx,:,:,0] = pd.DataFrame(x[:,:,0]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
    x_val2[idx,:,:,1] = pd.DataFrame(x[:,:,1]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values

    
x_test2 = np.ones(shape=x_test.shape)
for idx, x in enumerate(x_test):
    x_test2[idx,:,:,0] = pd.DataFrame(x[:,:,0]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
    x_test2[idx,:,:,1] = pd.DataFrame(x[:,:,1]).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
    

x_train_ecg = x_train2[:,:,:,1]
x_val_ecg = x_val2[:,:,:,1]
x_test_ecg = x_test2[:,:,:,1]





print('x_train shape:', x_train.shape)
print('x_test.shape:', x_test.shape)
print('x_val.shape:', x_val.shape)

print('x_train_ppg shape:', x_train_ppg.shape)

## Binary

In [ ]:
# binary classification
y_train_bin = y_train >= 4
y_test_bin = y_test >= 4
y_val_bin = y_val >= 4
y_train_bin2 = y_train >=7
y_val_bin2 = y_val>=7
y_test_bin2 = y_test>=7

## 3 class

In [ ]:
### 3 classes
# labels for y_train
y_train_class = []
for i in range(y_train.shape[0]):
    if y_train[i] <= 3.5:
        y_train_class.append((1,0,0))
    elif 3.5<y_train[i]<=6.5:
        y_train_class.append((0,1,0))
    else:
        y_train_class.append((0,0,1))
        
y_train_class = np.array(y_train_class, int)


# labels for y_val
y_val_class = []
for i in range(y_val.shape[0]):
    if y_val[i] <= 3.5:
        y_val_class.append((1,0,0))
    elif 3.5<y_val[i]<=6.5:
        y_val_class.append((0,1,0))
    else:
        y_val_class.append((0,0,1))
        
y_val_class = np.array(y_val_class, int)

# labels for y_test
y_test_class = []
for i in range(y_test.shape[0]):
    if y_test[i] <= 3.5:
        y_test_class.append([1,0,0])
    elif 3.5<y_test[i]<=6.5:
        y_test_class.append([0,1,0])
    else:
        y_test_class.append([0,0,1])
        
y_test_class = np.array(y_test_class, int)

## Sample weight

### 2 class (NRS>=4, NRS<4)

In [ ]:
# 2 class에 대한 sample weight
train_w_samp2 = np.ones(shape=(len(y_train),))
train_w_samp2[y_train_bin==0]= len(y_train) / np.sum(y_train_bin)
train_w_samp2[y_train_bin!=0]= len(y_train) / np.sum(~y_train_bin)

train_w_samp2_2 = np.ones(shape=(len(y_train),))
train_w_samp2_2[y_train_bin==0]= len(y_train) / np.sum(y_train_bin2)
train_w_samp2_2[y_train_bin!=0]= len(y_train) / np.sum(~y_train_bin2)

print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_train) / np.sum(y_train_bin), len(y_train) / np.sum(~y_train_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}'
      .format(len(y_train) / np.sum(y_train_bin2), len(y_train) / np.sum(~y_train_bin2)))


# 2 class에 대한 sample weight
val_w_samp2 = np.ones(shape=(len(y_val),))
val_w_samp2[y_val_bin==0]= len(y_val) / np.sum(y_val_bin)
val_w_samp2[y_val_bin!=0]= len(y_val) / np.sum(~y_val_bin)

val_w_samp2_2 = np.ones(shape=(len(y_val),))
val_w_samp2_2[y_val_bin==0]= len(y_val) / np.sum(y_val_bin2)
val_w_samp2_2[y_val_bin!=0]= len(y_val) / np.sum(~y_val_bin2)

print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_val) / np.sum(y_val_bin), len(y_val) / np.sum(~y_val_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}'
      .format(len(y_val) / np.sum(y_val_bin2), len(y_val) / np.sum(~y_val_bin2)))


# 2 class에 대한 sample weight
test_w_samp2 = np.ones(shape=(len(y_test),))
test_w_samp2[y_test_bin==0]= len(y_test) / np.sum(y_test_bin)
test_w_samp2[y_test_bin!=0]= len(y_test) / np.sum(~y_test_bin)

test_w_samp2_2 = np.ones(shape=(len(y_test),))
test_w_samp2_2[y_test_bin==0]= len(y_test) / np.sum(y_test_bin2)
test_w_samp2_2[y_test_bin!=0]= len(y_test) / np.sum(~y_test_bin2)

print('sample weight for no pain: {:.2f}, moderate pain: {:.2f}'
      .format(len(y_test) / np.sum(y_test_bin), len(y_test) / np.sum(~y_test_bin)))
print('sample weight for no pain: {:.2f}, severe pain: {:.2f}'
      .format(len(y_test) / np.sum(y_test_bin2), len(y_test) / np.sum(~y_test_bin2)))

### 3 class (NRS>=7, 7>NRS>=4, NRS<4)

In [ ]:
# 3 class에 대한 sample weight
train_w_samp3 = np.ones(shape=(len(y_train),))

train_w_samp3[y_train<4]= len(y_train)/np.sum(y_train<4)
train_w_samp3[(y_train>=4)&(y_train<7)]= len(y_train)/np.sum((4<=y_train)&(y_train<7))
train_w_samp3[y_train>=7]= len(y_train)/np.sum(y_train>=7)

print('train set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\n'
      .format(len(y_train)/np.sum(y_train<4),len(y_train)/np.sum((y_train>=4)&(y_train<7)),len(y_train)/np.sum(y_train>=7)))

# 3 class에 대한 sample weight
val_w_samp3 = np.ones(shape=(len(y_val),))

val_w_samp3[y_val<4]= len(y_val)/np.sum(y_val<4)
val_w_samp3[(y_val>=4)&(y_val<7)]= len(y_val)/np.sum((4<=y_val)&(y_val<7))
val_w_samp3[y_val>=7]= len(y_val)/np.sum(y_val>=7)

print('val set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\n'
      .format(len(y_val)/np.sum(y_val<4),len(y_val)/np.sum((y_val>=4)&(y_val<7)),len(y_val)/np.sum(y_val>=7)))

# 3 class에 대한 sample weight
test_w_samp3 = np.ones(shape=(len(y_test),))

test_w_samp3[y_test<4]= len(y_test)/np.sum(y_test<4)
test_w_samp3[(y_test>=4)&(y_test<7)]= len(y_test)/np.sum((4<=y_test)&(y_test<7))
test_w_samp3[y_test>=7]= len(y_test)/np.sum(y_test>=7)

print('test set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}'
      .format(len(y_test)/np.sum(y_test<4),len(y_test)/np.sum((y_test>=4)&(y_test<7)),len(y_test)/np.sum(y_test>=7)))

In [ ]:
# 3 class에 대한 sample weight
train_w_samp3 = np.ones(shape=(len(y_train),))

train_w_samp3[y_train<4]= len(y_train)/np.sum(y_train<4)
train_w_samp3[(y_train>=4)&(y_train<7)]= len(y_train)/np.sum((4<=y_train)&(y_train<7))
train_w_samp3[y_train>=7]= len(y_train)/np.sum(y_train>=7)


print('train set')
print('sample weight for class 1: {:.2f}, class 2: {:.2f}, class 3: {:.2f}\n'
      .format(len(y_train)/np.sum(y_train<4),len(y_train)/np.sum((y_train>=4)&(y_train<7)),len(y_train)/np.sum(y_train>=7)))


val_w_samp3 = np.ones(len(y_val))

test_w_samp3 = np.ones(shape=(len(y_test),))

# Settings

In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
# folder
nfold = 1  # 각각의 hyperparameter에 대해 k-fold 를 시행하고 평균을 구한다.
ntest = 100
rootdir = "preprocess6/NRS>=4_welch1/LSTM_3layers_sample-weighted_3class"

predirs = []
for root, dirs, files in os.walk(rootdir):  # 하위 대상들을 recursive 하게 긁어옴
    for filename in dirs:
        predirs.append(filename)

if not os.path.exists(rootdir):
    os.mkdir(rootdir)


# test_settings
test_settings_1, test_settings_2, test_settings_3 = [], [], []


# hyperparamters
#num_nodes = [64, 64, 64] #, 64, 64, 64]
#kernel_size = 10
pool_size = 2

#dense_node = 32
#dropout_rate = 0.2
learning_rate = 0.002

# hyperparamters pool
dropout_opts  = [0, 0.1, 0.2, 0.3, 0.4, 0.5] # dropout rate
dense_opts = [8, 16, 32, 64]
BATCH_SIZE = [512, 1024]


unit_opts = [4, 8, 16, 32, 64, 128, 256]

print('start making test settings...', end='', flush=True)
# test settings
for unit1 in unit_opts:
    for unit2 in unit_opts:
        for unit3 in unit_opts:
            for dense_node1 in dense_opts:
                for dense_node2 in dense_opts:
                    for dropout1 in dropout_opts:
                        for dropout2 in dropout_opts:
                            for dropout3 in dropout_opts:
                                for batch_size in BATCH_SIZE:
                                    test_settings_1.append([unit1, unit2, unit3, dense_node1, dense_node2, dropout1, dropout2, dropout3, batch_size])

print('done')

# RandomSearch

## binary

In [ ]:
from keras import metrics
# random search for hyperparameter
ntrial = 500
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_acc = []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    
    rnn_type = random.choice(['lstm','gru'])
        
    # test_setting
    unit1, unit2, unit3, dense_node1, dense_node2, dropout1, dropout2, dropout3, batch_size = test_setting_1
    
    
    # total LSTM layers of the model
    n_unit = random.choice([1,2,3])   

    if n_unit==1:
        units = [unit1]

    if n_unit==2:
        units = [unit1, unit2]
        
    if n_unit==3:
        units = [unit1, unit2, unit3]
        
        
    # final dense layers
    n_dense = random.choice([0,1])
    
    if n_dense ==0:
        dense_node1 = 0
        dropout2 = 0 
        dense_node2 = 0
        dropout3 = 0
    
    if n_dense ==1:
        dense_node2 = 0
        dropout3 = 0
        
    
    
    # 이번 옵션에 대한 결과 디렉토리
    odir_f = '{}_batch={},'.format(rnn_type, batch_size)    
    for unit in units:
        odir_f += 'unit{},'.format(unit)
    odir_f += 'dropout={},dnodes={},dropout={},dnodes={},dropout={}'.format(dropout1, dense_node1, dropout2, dense_node2, dropout3)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/weights.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:1"])
    with strategy.scope():
        
        # build a model - function api
        inp = Input(shape=(x_train.shape[1], x_train.shape[2]))

        if rnn_type == 'lstm':
            # lstm layer
            inp_lstm = inp
            for unit in units[:-1]:
                inp_lstm = LSTM(unit, return_sequences=True) (inp_lstm)
            out_lstm = LSTM(units[-1]) (inp_lstm)

        else:
            # gru layer
            inp_lstm = inp
            for unit in units[:-1]:
                inp_lstm = GRU(unit, return_sequences=True) (inp_lstm)
            out_lstm = GRU(units[-1]) (inp_lstm)            
            
            
        out_lstm = Dropout(dropout1) (out_lstm)

        if n_dense >= 1:
            out_lstm = Dense(dense_node1, activation='tanh') (out_lstm)
            out_lstm = Dropout(dropout2) (out_lstm)
            
        if n_dense >= 2:
            out_lstm = Dense(dense_node2, activation='tanh') (out_lstm)
            out_lstm = Dropout(dropout3) (out_lstm)            
            

        # output
        out = Dense(1, activation='sigmoid') (out_lstm)
        model = Model(inputs=[inp], outputs=[out])
        



        # model 학습 설정
        try:
            model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_train_ecg, y_train_bin, sample_weight=train_w_samp3, validation_data=(x_val_ecg, y_val_bin, val_w_samp3), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])
        except:
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue
            
            
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test_ecg).flatten()

    
    # acc 계산
    acc = metrics.Accuracy()
    acc.update_state(y_pred>=0.5, y_test_bin, sample_weight=test_w_samp3)
    acc_val = acc.result().numpy()
    test_acc.append(acc_val)
    
    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred, sample_weight=test_w_samp3)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)

    # auprc 
    precision, recall, _ = precision_recall_curve(y_test_bin, y_pred, sample_weight=test_w_samp3)
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)

    
    # rename
    os.rename(odir, rootdir+'/roc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_roc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_roc(max_idx), random_settings(max_idx)))


## age+gender

In [ ]:
import pandas as pd
import pickle

print('loading train...', flush=True, end='')

# x를 loading해서 (batch_size, step, channel)
input_path = '../dataset/preprocess4/input2/'

gender_train = np.load(input_path+'gender_train.npz', allow_pickle=True)['arr_0']
gender_test = np.load(input_path+'gender_test.npz', allow_pickle=True)['arr_0']
gender_val = np.load(input_path+'gender_val.npz', allow_pickle=True)['arr_0']

age_train = np.load(input_path+'age_train.npz', allow_pickle=True)['arr_0']
age_test = np.load(input_path+'age_test.npz', allow_pickle=True)['arr_0']
age_val = np.load(input_path+'age_val.npz', allow_pickle=True)['arr_0']
print('done', flush=True)


input_path = '../dataset/preprocess4/welch/'
train_mask = pickle.load(open(input_path+'train_mask_pacu', 'rb')) + pickle.load(open(input_path+'train_mask_preop', 'rb'))
val_mask = pickle.load(open(input_path+'val_mask_pacu', 'rb')) + pickle.load(open(input_path+'val_mask_preop', 'rb'))
test_mask = pickle.load(open(input_path+'test_mask_pacu', 'rb')) + pickle.load(open(input_path+'test_mask_preop', 'rb'))

print(len(train_mask), np.sum(train_mask), len(x_train_ecg))

In [ ]:
len(x_train_ecg), len(gender_train), (np.sum(train_mask))

In [ ]:
# input, sample weight settings
train_w_samp = train_w_samp3
val_w_samp = val_w_samp3
test_w_samp = test_w_samp3

# 
gender_train = gender_train[train_mask]
gender_val = gender_val[val_mask]
gender_test = gender_test[test_mask]

age_train = age_train[train_mask]
age_val = age_val[val_mask]
age_test = age_test[test_mask]


agender_train = np.array([[age_train[i], gender_train[i]] for i in range(len(age_train))])
agender_val = np.array([[age_val[i], gender_val[i]] for i in range(len(age_val))])
agender_test = np.array([[age_test[i], gender_test[i]] for i in range(len(age_test))])

x_trains = [x_train_ecg, agender_train]
x_vals = [x_val_ecg, agender_val]
x_tests = [x_test_ecg, agender_test]

In [ ]:
from keras import metrics
# random search for hyperparameter
ntrial = 700
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_acc = []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    
    rnn_type = random.choice(['lstm','gru'])
        
    # test_setting
    unit1, unit2, unit3, dense_node1, dense_node2, dropout1, dropout2, dropout3, batch_size = test_setting_1
    
    
    # total LSTM layers of the model
    n_unit = random.choice([1,2,3])   

    if n_unit==1:
        units = [unit1]

    if n_unit==2:
        units = [unit1, unit2]
        
    if n_unit==3:
        units = [unit1, unit2, unit3]
        
        
    # final dense layers
    n_dense = random.choice([0,1])
    
    if n_dense ==0:
        dense_node1 = 0
        dropout2 = 0 
        dense_node2 = 0
        dropout3 = 0
    
    if n_dense ==1:
        dense_node2 = 0
        dropout3 = 0
        
    
    
    # 이번 옵션에 대한 결과 디렉토리
    odir_f = '{}_batch={},'.format(rnn_type, batch_size)    
    for unit in units:
        odir_f += 'unit{},'.format(unit)
    odir_f += 'dropout={},dnodes={},dropout={}'.format(dropout1, dense_node1, dropout2)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/weights.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:1"])
    with strategy.scope():
        
        # build a model - function api
        inp_fnn = Input(shape=(agender_train.shape[1],))
        inp = Input(shape=(x_train_ecg.shape[1], x_train_ecg.shape[2]))
        inp_lstm = inp
        
        out_fnn = inp_fnn
        out_fnn = Activation('sigmoid') (out_fnn)
        

        if rnn_type == 'lstm':
            # lstm layer
            for unit in units[:-1]:
                inp_lstm = LSTM(unit, return_sequences=True) (inp_lstm)
            out_lstm = LSTM(units[-1]) (inp_lstm)

        else:
            # gru layer
            for unit in units[:-1]:
                inp_lstm = GRU(unit, return_sequences=True) (inp_lstm)
            out_lstm = GRU(units[-1]) (inp_lstm)            
            
            
        out_lstm = Dropout(dropout1) (out_lstm)
        out = concatenate([out_fnn, out_lstm])

        if n_dense >= 1:
            out = Dense(dense_node1, activation='tanh') (out)
            out = Dropout(dropout2) (out)         
            

        # output
        out = Dense(1, activation='sigmoid') (out)
        model = Model(inputs=[inp, inp_fnn], outputs=[out])


        # model 학습 설정
        try:
            model.compile(loss='binary_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_trains, y_train_bin, sample_weight=train_w_samp3, validation_data=(x_vals, y_val_bin, val_w_samp3), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')])
        except Exception as e:
            print(e)
            os.rmdir(odir)
            #os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            itrial -= 1
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue
            
            
    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_tests).flatten()

    
    # acc 계산
    acc = metrics.Accuracy()
    acc.update_state(y_pred>=0.5, y_test_bin, sample_weight=test_w_samp3)
    acc_val = acc.result().numpy()
    test_acc.append(acc_val)
    
    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred, sample_weight=test_w_samp3)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)

    # auprc 
    precision, recall, _ = precision_recall_curve(y_test_bin, y_pred, sample_weight=test_w_samp3)
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)

    
    # rename
    os.rename(odir, rootdir+'/roc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_roc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_roc(max_idx), random_settings(max_idx)))


## 3 class

In [ ]:
from keras import metrics
from keras.layers import LeakyReLU, ReLU
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# random search for hyperparameter
ntrial = 200
train_errs, val_errs = [] ,[]
test_roc, test_prc = [], []
test_rmse, test_acc = [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    

    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2

        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={}, c1={}, c2={}, c3={}, c4={}, c1filts={}, c2filts={}, c3filts={}, c4filts={}, conv_double={}, globalpool_opt={}, dropout={}, dnodes={}, dropout={}'.format(batch_size, num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4, conv_double, globalpool_opt, dropout_cnn, dense_node, dropout_fc)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/model.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:2", "/gpu:3"])
    with strategy.scope():
        # build a model
        model = Sequential()

        conv_act = True
        if conv_act:
            act = 'relu'
        else:
            act = None

        # c1 layer
        if conv_double:
            model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same'))
        model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))

        # c2 layer
        if conv_double:
            model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same'))
        model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c3 layer
        if conv_double:
            model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same'))
        model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c4 layer
        if conv_double:
            model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same'))
        model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            model.add(GlobalMaxPool1D())
        elif globalpool_opt == 'ave':
            model.add(GlobalAveragePooling1D())
            
            
        if dense_node != 0:
            model.add(Dropout(dropout_cnn))
            model.add(Dense(dense_node, activation='tanh'))
        model.add(Dropout(dropout_fc))
        model.add(Dense(3, activation='softmax'))

        try:
            # model 학습 설정
            model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=learning_rate), metrics=["acc", tf.keras.metrics.AUC()])
            hist = model.fit(x_train, y_train_class, validation_data=(x_val, y_val_class), epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                                EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])
        except:
            os.rename(odir,rootdir+'/error_{}'.format(odir_f))
            test_roc.append(0)
            test_acc.append(0)
            test_prc.append(0)
            train_errs.append(-1)
            val_errs.append(-1)
            continue

    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test)

    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve((y_test_class[:,1]+y_test_class[:,2])>=1, y_pred[:,1]+y_pred[:,2])
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_roc.append(roc_auc)
    
    # auprc 
    precision, recall, _ = precision_recall_curve((y_test_class[:,1]+y_test_class[:,2])>=1, y_pred[:,1]+y_pred[:,2])
    prc_auc = auc(recall, precision)
    test_prc.append(prc_auc)
    
    # acc 계산
    l_test = np.argmax(y_test_class, axis=1)
    l_pred = np.argmax(y_pred, axis=1)
    acc_val = accuracy_score(l_test, l_pred)
    test_acc.append(acc_val)
    
    # rename
    os.rename(odir, rootdir+'/auc{:.4f}_prc{:.4f}_{}_acc{:.2f}'.format(roc_auc, prc_auc, odir_f, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_auc(max_idx), random_settings(max_idx)))

## Regression

In [ ]:
from keras import metrics
# random search for hyperparameter
ntrial = 100
train_errs, val_errs = [], []
test_auc, test_rmse, test_acc = [], [], []
random_settings = []


for itrial in range(ntrial):
    # grid search
    # test_setting = test_settings[itrial]

    # random search
    print('random search {}/{}'.format(itrial, ntrial))
    test_setting_1 = random.choice(test_settings_1)
    test_setting_2 = random.choice(test_settings_2)
    

    # test_setting
    num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4 = test_setting_1
    dense_node, dropout_cnn, dropout_fc, globalpool_opt, batch_size, conv_double = test_setting_2

        

    # 이번 옵션에 대한 결과 디렉토리
    odir_f = 'batch={}, c1={}, c2={}, c3={}, c4={}, c1filts={}, c2filts={}, c3filts={}, c4filts={}, conv_double={}, globalpool_opt={}, dropout={}, dnodes={}, dropout={}'.format(batch_size, num_l1, num_l2, num_l3, num_l4, kernel_l1, kernel_l2, kernel_l3, kernel_l4, conv_double, globalpool_opt, dropout_cnn, dense_node, dropout_fc)
    random_settings.append(odir_f)
    
    odir = rootdir + '/' + odir_f
    if not os.path.exists(odir):
        os.mkdir(odir)

    weightcache = "{}/model.hdf5".format(odir)        

    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    with strategy.scope():
        # build a model
        model = Sequential()

        conv_act = True
        if conv_act:
            act = 'relu'
        else:
            act = None

        # c1 layer
        if conv_double:
            model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same'))
        model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))

        # c2 layer
        if conv_double:
            model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same'))
        model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c3 layer
        if conv_double:
            model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same'))
        model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))
        
        
        # c4 layer
        if conv_double:
            model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same'))
        model.add(Conv1D(filters=num_l4, kernel_size=kernel_l4, padding='same', activation=act))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size=pool_size))


        # global이냐 flatten이냐는 따로 모델 나눠야 할듯
        if globalpool_opt == 'max':
            model.add(GlobalMaxPool1D())
        elif globalpool_opt == 'ave':
            model.add(GlobalAveragePooling1D())
            
            
        if dense_node != 0:
            model.add(Dropout(dropout_cnn))
            model.add(Dense(dense_node, activation='sigmoid'))
        model.add(Dropout(dropout_fc))
        model.add(Dense(1, activation='sigmoid'))


        # model 학습 설정
        model.compile(loss='mse', optimizer=Adam(lr=learning_rate), metrics=["mean_absolute_error", tf.keras.metrics.AUC()])
        hist = model.fit(x_train, y_train/10, validation_split=0.1, epochs=100, batch_size=batch_size, #class_weight={0:1, 1:3}, 
                                callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weightcache, verbose=1, save_best_only=True),
                                            EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])


    # 모델의 아키텍처 및 구조 저장
    open(odir+"/model.json", "wt").write(model.to_json())

    # test set에 대한 y_pred 계산
    model.load_weights(weightcache)  # fit 함수는 마지막 epoch의 결과를 리턴하기 때문에 best 결과를 다시 읽어들어야함
    y_pred = model.predict(x_test).flatten()

    # auroc 계산
    false_positive_rate, true_positive_rate, threshold = roc_curve(y_test_bin, y_pred)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    test_auc.append(roc_auc)
    # RMSE 계산
    model_err = metrics.RootMeanSquaredError()
    model_err.update_state(y_test, y_pred)
    rmse_val = model_err.result().numpy()
    test_rmse.append(rmse_val)
    # acc 계산
    acc_val = np.mean((y_pred*10>=5)==y_test_bin)
    test_acc.append(acc_val)
    # rename
    os.rename(odir, rootdir+'/auc{:.4f}_{}_rmse{:.4f}_acc{:.2f}'.format(roc_auc, odir_f, rmse_val, acc_val))

    # train 과정에서의 err
    train_err = min(hist.history['loss'])
    val_err = min(hist.history['val_loss'])

    val_errs.append(val_err)
    train_errs.append(train_err)


    tf.keras.backend.clear_session()


max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}, info: {}'.format(test_auc(max_idx), random_settings(max_idx)))

# Results

## 1D-CNN model 1
[input]-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(global maxpool)-dropout-(dense)-dropout-[output]
<br>or conv-conv-bn-maxpool

### Best Result

In [ ]:
max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}\ninfo: {}'.format(test_auc[max_idx], random_settings[max_idx]))

### Top 10 Result

In [ ]:
# top 10 model
topid= sorted(range(len(test_auc)),key= lambda i: test_auc[i])[-10:]

for i in range(10):
    print('Top {} Model: roc {:.4f}   train mse {:.4f}  val mse {:4f}'.format(i+1, np.array(test_auc)[topid[9-i]], np.array(train_errs)[topid[9-i]], np.array(val_errs)[topid[9-i]]))
    print(' {}\n'.format(np.array(random_settings[topid[9-i]])))

    #np.array(test_auc)[topid], np.array(random_settings)[topid]

## 1D-CNN model 2
[input]-(conv1-bn-maxpool)-(conv1-bn-maxpool)-(global max or ave or flatten)-dropout-(dense)-dropout-[output]

In [ ]:
max_idx = test_auc.index(max(test_auc))
print('\nBest Model roc:{:.4f}  rmse:{:.4f}  acc:{:.2f}\n: {}'.format(test_auc[max_idx], test_rmse[max_idx], test_acc[max_idx], random_settings[max_idx]))

In [ ]:
# top 10 model
topid= sorted(range(len(test_auc)),key= lambda i: test_auc[i])[-10:]

for i in range(10):
    print('Top {} Model: roc {:.4f}  rmse:{:.4f}  acc:{:.2f},  train mse {:.4f}  val mse {:4f}'.format(i+1, np.array(test_auc)[topid[9-i]],np.array(test_rmse)[topid[9-i]], np.array(test_acc)[topid[9-i]], np.array(train_errs)[topid[9-i]], np.array(val_errs)[topid[9-i]]))
    print(' {}\n'.format(np.array(random_settings[topid[9-i]])))

    #np.array(test_auc)[topid], np.array(random_settings)[topid]

In [ ]:
def build_model(num_l1=256, kernel_l1 = 10, bool_flatten=False, dropout=0.2):
    #strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    #with strategy.scope():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    model = Sequential()
    #num_l2=64, num_l3=64, kernel_l2=3, kernel_l3=3
    num_l2=64
    num_l3=64
    kernel_l2=3
    kernel_l3=3

    #for (num_node, kernel_size) in conv_layers:
    #    model.add(Conv1D(filters=num_node, kernel_size=kernel_size, padding='valid'))
    #    model.add(BatchNormalization())
    #    model.add(MaxPooling1D(pool_size=2))

    # Conv Layer 1
    model.add(Conv1D(filters=num_l1, kernel_size=kernel_l1, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # Conv Layer 2
    model.add(Conv1D(filters=num_l2, kernel_size=kernel_l2, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # Conv Layer 3
    model.add(Conv1D(filters=num_l3, kernel_size=kernel_l3, padding='valid'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2)) 

    # FC layer 이전의 작업
    if bool_flatten:
        model.add(Flatten())
    else:
        model.add(GlobalMaxPool1D())

    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])


return model